[deps]
- CairoMakie
- Makie
- MakieCore

In [ ]:
using MakieCore
import MakieCore: convert_arguments 

# This part would be the extension package (calls like lines, scatter, etc.)
using CairoMakie

First, a mini vesion of a NamedTrajectory

In [ ]:
struct NamedVariable{R}
    data::AbstractVector{R}
    components::NamedTuple{
        cnames, <:Tuple{Vararg{AbstractVector{Int}}}
    } where cnames
    names::Tuple{Vararg{Symbol}}
end

function NamedVariable(nt::NamedTuple{S, <:Tuple{Vararg{AbstractVector{R}}}}) where {S, R}
    data = vcat(nt...)
    names = keys(nt)
    total = 1
    indices = []
    for i in map(length, nt)
        push!(indices, total:total+i-1)
        total += i
    end
    return NamedVariable(data, NamedTuple{names}(indices), names)
end

function Base.getproperty(nv::NamedVariable, name::Symbol)
    if name in getfield(nv, :names)
        idx = getfield(nv, :components)[name]
        return nv.data[idx]
    else
        return getfield(nv, name)
    end
end

function Base.getindex(nv::NamedVariable, name::Symbol)
    return getproperty(nv, name)
end

convert_arguments:
- a recursive call is needed after you get a list of points
- a plot spec is one way to make a grid, but it's a bit premature

after convert arguments exists, recipes are the way

In [ ]:
function convert_arguments(
    P::MakieCore.PointBased, 
    var::NamedVariable{<:Any},
    name::Symbol
)
    positions = [(t, b) for (t, b) in zip(1:length(var[name]), var[name])]
    return convert_arguments(P, positions)
end

# define the default plot type (preserve `plot` calls)
MakieCore.plottype(::NamedVariable, ::Symbol) = Lines

In [ ]:
import Makie.SpecApi as S

function convert_arguments(
    P::Type{Plot{MakieCore.plot}},
    v::NamedVariable{<:Any},
    labels::AbstractVector{Symbol}
    # label_kwargs::Dict{Symbol, Any} = Dict()
)
    axes = [
        S.Axis(plots=[S.Lines(1:length(v[l]), v[l])]) for l in labels, _ in 1:1
    ]
    return S.GridLayout(
        axes,    )
    # return PlotSpec[S.Lines(1:length(v[l]), v[l]) for l in labels]
end

Here are all the keyword arguments that S.GridLayout accepts (_unclear what the allowed
types are_)

```julia
S.GridLayout([...],
    colsizes = [Auto(), Auto(), 300],
    rowsizes = [Relative(0.4), Relative(0.6)],
    colgaps,
    rowgaps,
    alignmode,
    halign,
    valign,
    tellheight,
    tellwidth,
)
```

Test the features

In [ ]:
v = NamedVariable((a=[1, 2, 3], b=[4, 4, 4]))

In [ ]:
convert_arguments(Lines, v, :a)

Plot using convert args

In [ ]:
f = Figure()
ax = Axis(f[1, 1])
plot!(ax, v, :a)
lines!(ax, v, :b)
f

In [ ]:
lines(v, :a, linestyle=:dash, linewidth=10)

Plot using the spec (experimental)

In [ ]:
f = Figure()

# works
plot(f[1,2], v, [:a, :b])

# won't work
# ax = Axis(f[1, 1])
# plot(ax, v, [:a, :b])

f

In [ ]:
f = Figure()
ax = Axis(f[1:2, 1])
plot!(ax, v, [:a, :b])
f

TODO: Section for recipes


- What features do we need? 
- Transformations, labels, 

- Every Block such as Axis, Legend, Colorbar, etc. can be themed by using its type name as a key in your theme.

In [ ]:
# dark_latexfonts = merge(theme_dark(), theme_latexfonts())
# lines_theme = Theme(
#     Lines = (
#         linewidth = 4,
#         linestyle = :dash,
#     ),
#     Axis = (
#         leftspinevisible = false,
#     ),
# )

# set_theme!(dark_latexfonts)
# with_theme(dark_latexfonts, fontsize = 25) do
#     example_plot()
# end

# You can use the keys rowgap and colgap to change the default grid layout gaps.

# The theme given in the body of the @recipe invocation is inserted into a specialization
# of default_theme which inserts the theme into any scene that plots MyPlot.

# The do scene closure is just a function that returns our default attributes

In [ ]:
@recipe(PlotVariable, x, y, z) do scene
    Theme(
        Lines = (
            linewidth = 4,
            linestyle = :dash,
        ),
        Axis = (
            leftspinevisible = false,
        ),
    )
    Attributes(
        labels = false,
    )
end

function Makie.plot!(pv::PlotVariable)
    lines!(pv, rand(10), color = pv.plot_color)
    pv
end